# **BirdCLEF 2025 Data Preprocessing Notebook**

In [ ]:
import os
import cv2
import math
import time
import librosa
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import os
import numpy as np
from datetime import datetime
import pytz
import torch
import warnings
warnings.filterwarnings("ignore")

from joblib import Parallel, delayed


from module import config_lib, utils_lib

In [2]:
class DatasetConfig:
    def __init__(self, kaggle_notebook=False, debug=False):
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.model_path = '/kaggle/input/birdclef-2025-0330'
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.models_dir = "../models/" # 全modelの保存先
            self.model_path = self.models_dir # 各モデルの保存先．学習時に動的に変更．
            self.RAW_DIR = '../data/raw/'
            self.PROCESSED_DIR = '../data/processed/'


        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5.0 # 推論時のウィンドウサイズ
        self.TARGET_DURATION = 5 # データセット作成時のウィンドウサイズ
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 16
        self.N_MELS = 148
        self.FMIN = 20
        self.FMAX = 16000
        self.N_MAX = 50 if self.debug else None        
        self.N_JOBS = 16  # 並列処理のスレッド数 16くらいでいい
        self.LOAD_ENGINE = 'torchaudio'  # librosa or torchaudio
        self.SKIP_RESIZE = False  # resizeしないならTrue
        self.seed = 42
            

In [3]:
config = DatasetConfig(kaggle_notebook=False, debug=False)

In [4]:
utils_lib.set_seed(config.seed)

In [5]:
print(f"Debug mode: {'ON' if config.debug else 'OFF'}")
print(f"Max samples to process: {config.N_MAX if config.N_MAX is not None else 'ALL'}")

print("Loading taxonomy data...")
taxonomy_df = pd.read_csv(f'{config.RAW_DIR}/taxonomy.csv')
species_class_map = dict(zip(taxonomy_df['primary_label'], taxonomy_df['class_name']))

print("Loading training metadata...")
train_df = pd.read_csv(f'{config.RAW_DIR}/train.csv')

Debug mode: OFF
Max samples to process: ALL
Loading taxonomy data...
Loading training metadata...


In [6]:
label_list = sorted(train_df['primary_label'].unique())
label_id_list = list(range(len(label_list)))
label2id = dict(zip(label_list, label_id_list))
id2label = dict(zip(label_id_list, label_list))

print(f'Found {len(label_list)} unique species')
working_df = train_df.copy()
working_df['target'] = working_df.primary_label.map(label2id)
working_df['filepath'] = config.RAW_DIR + '/train_audio/' + working_df.filename
working_df['samplename'] = working_df.filename.map(lambda x: x.split('/')[0] + '-' + x.split('/')[-1].split('.')[0])
working_df['class'] = working_df.primary_label.map(lambda x: species_class_map.get(x, 'Unknown'))
working_df["crop_strategy"] = "center"
total_samples = min(len(working_df), config.N_MAX or len(working_df))
print(f'Total samples to process: {total_samples} out of {len(working_df)} available')
print(f'Samples by class:')
print(working_df['class'].value_counts())

Found 206 unique species
Total samples to process: 28564 out of 28564 available
Samples by class:
class
Aves        27648
Amphibia      583
Mammalia      178
Insecta       155
Name: count, dtype: int64


In [7]:
# Fabio A. Sarria-Sの音声は最初にInsectaの鳴き声があるため，最初をcropしたい
# crop_starategyを'head'に変更する

# 1. "Fabio A. Sarria-S" の filename を抽出
fabio_filenames = train_df.loc[
    train_df['author'] == "Fabio A. Sarria-S", 'filename'
].tolist()

# 2. working_df の crop_strategy を 'head' に更新
working_df['crop_strategy'] = working_df.get('crop_strategy', 'center')  # 初期化（なければcenter）

working_df.loc[
    working_df['filename'].isin(fabio_filenames),
    'crop_strategy'
] = 'head'

In [13]:
# rareなラベルに対し，multi-cropを行うためのflag設定

# rareなラベルを抽出（例：50未満）
label_counts = working_df['primary_label'].value_counts().rename_axis("label").reset_index(name="sample_count")
rare_labels = label_counts[label_counts['sample_count'] < 50]['label'].tolist()

# 初期値は0
working_df['n_augment'] = 0
working_df['multi_crop'] = False

# rare種に対して必要なaugment数を計算して割り当て
for rare_label in rare_labels:
    base_rows = working_df[working_df['primary_label'] == rare_label]
    n_exist = len(base_rows)
    n_needed = 50 - n_exist

    n_aug_per_sample = math.ceil(n_needed / n_exist)

    # フラグとaugment数をセット
    working_df.loc[
        working_df['primary_label'] == rare_label, 'multi_crop'
    ] = True
    working_df.loc[
        working_df['primary_label'] == rare_label, 'n_augment'
    ] = n_aug_per_sample
    

In [8]:
# crop_strategyに基づいて音声データを切り出す
def crop_audio(audio_data: np.ndarray, target_samples: int, strategy='center'):
    total_samples = len(audio_data)

    if total_samples < target_samples:
        n_copy = math.ceil(target_samples / total_samples)
        audio_data = np.concatenate([audio_data] * n_copy)
        total_samples = len(audio_data)

    if strategy == 'head':
        # 1秒遅らせて開始（ただし収まらない場合は0から）
        buffer = int(1.0 * config.FS)
        start_idx = min(buffer, total_samples - target_samples)
    elif strategy == 'tail':
        start_idx = total_samples - target_samples
    elif strategy == 'center':
        start_idx = total_samples // 2 - target_samples // 2
    elif strategy == 'random':
        max_start = total_samples - target_samples
        start_idx = np.random.randint(0, max_start + 1)
    elif isinstance(strategy, (float, int)):
        start_idx = int(strategy * config.FS)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    start_idx = max(0, min(start_idx, total_samples - target_samples))
    end_idx = start_idx + target_samples
    return audio_data[start_idx:end_idx]

In [9]:
def audio2melspec(audio_data):
    if np.isnan(audio_data).any():
        mean_signal = np.nanmean(audio_data)
        audio_data = np.nan_to_num(audio_data, nan=mean_signal)

    mel_spec = librosa.feature.melspectrogram(
        y=audio_data,
        sr=config.FS,
        n_fft=config.N_FFT,
        hop_length=config.HOP_LENGTH,
        n_mels=config.N_MELS,
        fmin=config.FMIN,
        fmax=config.FMAX,
        power=2.0
    )

    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    mel_spec_norm = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min() + 1e-8)
    
    return mel_spec_norm

In [ ]:
# 並列化

def process_row(row):
    try:
        audio_data, _ = librosa.load(row.filepath, sr=config.FS)
        target_samples = int(config.TARGET_DURATION * config.FS)

        strategy = row.crop_strategy
        try:
            strategy = float(strategy)
        except ValueError:
            pass

        mel_list = []
        name_list = []

        if row.get('multi_crop', False):
            n_crops = int(row.get('n_augment', 1))
            total_samples = len(audio_data)
            interval = max((total_samples - target_samples) // (n_crops + 1), 1)

            for i in range(n_crops):
                start_idx = i * interval
                clip = audio_data[start_idx: start_idx + target_samples]
                if len(clip) < target_samples:
                    clip = np.pad(clip, (0, target_samples - len(clip)), mode='constant')
                mel = audio2melspec(clip)
                if mel.shape != config.TARGET_SHAPE:
                    mel = cv2.resize(mel, config.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)
                mel_list.append(mel.astype(np.float32))
                name_list.append(f"{row.samplename}_crop{i}")
        else:
            clip = crop_audio(audio_data, target_samples, strategy=strategy)
            if len(clip) < target_samples:
                clip = np.pad(clip, (0, target_samples - len(clip)), mode='constant')
            mel = audio2melspec(clip)
            if mel.shape != config.TARGET_SHAPE:
                mel = cv2.resize(mel, config.TARGET_SHAPE, interpolation=cv2.INTER_LINEAR)
            mel_list.append(mel.astype(np.float32))
            name_list.append(row.samplename)

        return list(zip(name_list, mel_list)), None

    except Exception as e:
        return None, (row.filepath, str(e))


In [11]:
results = Parallel(n_jobs=config.N_JOBS)(
    delayed(process_row)(row) for _, row in working_df.iloc[:total_samples].iterrows()
)

# 結果の整理
all_bird_data = {name: spec for name, spec, err in results if name is not None}
errors = [err for name, spec, err in results if err is not None]


In [12]:
# 4mins
import os
import numpy as np
import pickle
import csv
from datetime import datetime
import pytz

# === JST時刻でディレクトリ作成 ===
jst = pytz.timezone('Asia/Tokyo')
now = datetime.now(jst)
timestamp = now.strftime("%Y%m%d_%H%M")
output_dir = os.path.join(config.PROCESSED_DIR, f"melspec_{timestamp}")
os.makedirs(output_dir, exist_ok=True)

# === 1. melスペクトログラムの保存 ===
output_path = os.path.join(output_dir, "birdclef2025_melspec_5sec_256_256.npy")

wrapped_array = np.array(all_bird_data, dtype=object)

with open(output_path, 'wb') as f:
    pickle.dump(wrapped_array, f, protocol=5)

print(f"\n✅ Mel-spectrograms saved to: {output_path}")
print(f"📦 File size: {os.path.getsize(output_path) / (1024 ** 2):.2f} MB")
print(f"📐 Example shape: {next(iter(all_bird_data.values())).shape}")

# === 2. configの保存 ===
config_path = os.path.join(output_dir, "config.csv")

# オブジェクトの属性を key-value に変換（__dict__ で取得）
config_dict = {k: v for k, v in vars(config).items() if not k.startswith("__")}

with open(config_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["key", "value"])
    for key, value in config_dict.items():
        writer.writerow([key, value])

print(f"📝 Config saved to: {config_path}")


✅ Mel-spectrograms saved to: ../data/processed/melspec_20250411_1524/birdclef2025_melspec_5sec_256_256.npy
📦 File size: 7143.48 MB
📐 Example shape: (256, 256)
📝 Config saved to: ../data/processed/melspec_20250411_1524/config.csv


In [14]:
spectrograms = np.load("../data/processed/mel_0411/birdclef2025_melspec_5sec_256_256.npy", allow_pickle=True).item()


In [15]:
spectrograms

{'1139490-CSA36385': array([[1.7118777e-04, 2.0463347e-01, 2.1627998e-01, ..., 0.0000000e+00,
         0.0000000e+00, 1.4685783e-01],
        [1.2166925e-02, 2.2783673e-01, 2.3946418e-01, ..., 0.0000000e+00,
         0.0000000e+00, 1.4746909e-01],
        [3.1053832e-02, 2.6436949e-01, 2.7596697e-01, ..., 0.0000000e+00,
         0.0000000e+00, 1.4843149e-01],
        ...,
        [4.1276175e-01, 5.0868046e-01, 4.8000607e-01, ..., 4.6040016e-01,
         4.7328359e-01, 4.6689528e-01],
        [2.2843847e-01, 1.9758661e-01, 1.8644863e-01, ..., 1.7883313e-01,
         1.8383743e-01, 1.8558300e-01],
        [1.1136827e-01, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
         0.0000000e+00, 6.9116955e-03]], dtype=float32),
 '1139490-CSA36389': array([[0.        , 0.12203522, 0.05301917, ..., 0.18614925, 0.13789214,
         0.16374509],
        [0.        , 0.14120689, 0.07242333, ..., 0.19972037, 0.15693086,
         0.17404364],
        [0.        , 0.17139205, 0.10297457, ..., 0.22

In [26]:
working_df[working_df["secondary_labels"] != "['']"]

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license,target,filepath,samplename,class,crop_strategy
9,126247,"['65448', '22976', '476538']",['advertisement call'],126247/XC941297.ogg,XC,3.5,https://xeno-canto.org/941297,9.0465,-79.3024,Leptodactylus insularum,Spotted Foam-nest Frog,Chris Harrison,cc-by-nc-sa 4.0,3,../data/raw//train_audio/126247/XC941297.ogg,126247-XC941297,Amphibia,center
185,22973,['compau'],"['advertisement call', 'chorus']",22973/XC892932.ogg,XC,4.5,https://xeno-canto.org/892932,10.6144,-61.2238,Leptodactylus fuscus,Whistling Grass Frog,Chris Harrison,cc-by-nc-sa 4.0,14,../data/raw//train_audio/22973/XC892932.ogg,22973-XC892932,Amphibia,center
453,476538,['65448'],['advertisement call'],476538/XC926710.ogg,XC,4.5,https://xeno-canto.org/926710,9.0709,-79.2983,Elachistocleis panamensis,Panama Humming Frog,Chris Harrison,cc-by-nc-sa 4.0,28,../data/raw//train_audio/476538/XC926710.ogg,476538-XC926710,Amphibia,center
689,65349,['65547'],['advertisement call'],65349/XC941283.ogg,XC,4.5,https://xeno-canto.org/941283,8.6211,-80.1393,Dendropsophus ebraccatus,Harlequin Treefrog,Chris Harrison,cc-by-nc-sa 4.0,42,../data/raw//train_audio/65349/XC941283.ogg,65349-XC941283,Amphibia,center
741,65448,['22976'],['advertisement call'],65448/XC941294.ogg,XC,4.5,https://xeno-canto.org/941294,9.0416,-79.3026,Engystomops pustulosus,Túngara Frog,Chris Harrison,cc-by-nc-sa 4.0,45,../data/raw//train_audio/65448/XC941294.ogg,65448-XC941294,Amphibia,center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28500,ywcpar,['orcpar'],['song'],ywcpar/XC540814.ogg,XC,4.5,https://xeno-canto.org/540814,9.0095,-79.4684,Amazona ochrocephala,Yellow-crowned Parrot,Kent Livezey,cc-by-nc-sa 4.0,205,../data/raw//train_audio/ywcpar/XC540814.ogg,ywcpar-XC540814,Aves,center
28501,ywcpar,"['yehcar1', 'orcpar', 'trokin']",['song'],ywcpar/XC540870.ogg,XC,4.5,https://xeno-canto.org/540870,9.0095,-79.4684,Amazona ochrocephala,Yellow-crowned Parrot,Kent Livezey,cc-by-nc-sa 4.0,205,../data/raw//train_audio/ywcpar/XC540870.ogg,ywcpar-XC540870,Aves,center
28508,ywcpar,"['neocor', 'trokin']",['song'],ywcpar/XC558007.ogg,XC,3.5,https://xeno-canto.org/558007,9.0095,-79.4684,Amazona ochrocephala,Yellow-crowned Parrot,Kent Livezey,cc-by-nc-sa 4.0,205,../data/raw//train_audio/ywcpar/XC558007.ogg,ywcpar-XC558007,Aves,center
28511,ywcpar,['orcpar'],['song'],ywcpar/XC569601.ogg,XC,4.5,https://xeno-canto.org/569601,9.0095,-79.4684,Amazona ochrocephala,Yellow-crowned Parrot,Kent Livezey,cc-by-nc-sa 4.0,205,../data/raw//train_audio/ywcpar/XC569601.ogg,ywcpar-XC569601,Aves,center
